In [ ]:
import psycopg2 as ps
import ipyleaflet as ipyl
import ipywidgets as ipyw
import pandas as pd
import json



# Preliminary works:
 To extract OpenStreetMap data that we want to display here, 
  we created filter following transnet project as below:
  1. All power nodes/ways of relations tagged with power=*
  2. All relations tagged with route=power
  3. All ways and its corresponding nodes tagged with power=*


Afterwards, we merge all extracted data before converts it using osm2pgsql to postGIS-enabled PostgreSQL databases.

# Database Connection and Query

This section handle database connection to our postgreSQL database.
Since ipyleaflet working with GeoJSON format, we use query database that convert our extracted database to JSON format.

## Note:
User needs to enter database name, user and password information in the following cell, respective variables.

In [ ]:
db_name= "gis"
db_user = ""
db_password = ""

## Database Connection

Here the database connection is done in the following cell.


In [ ]:
conn_string = "dbname='"+db_name+"' user='" + db_user + "' password='" + db_password + "' host='localhost'"
conn = ps.connect(conn_string)
cur = conn.cursor()

## Query Database

Next, the database is queried for its planet_osm_line and planet_osm_point. As power related information we chose next columns to show in the map.

In [ ]:
columns = """
        operator,
        name,
        highway,
        amenity,
        public_transport,
        railway,
        power,
        route
"""

### Create Queries

Query string for line objects. The query takes the line objects, converts its WebMercator(EPSG:3857) coordinates to WGS 84(EPSG:4326) and create GeoJSON structured string using all the information, to be able to use it in ipyleaflet map.

In [ ]:
query_json_line = """
SELECT row_to_json(fc)
 FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
 FROM (SELECT 'Feature' As type
    , ST_AsGeoJSON(ST_Transform(lg.way, 4326))::json As geometry
    , row_to_json((SELECT l FROM (SELECT {}) As l
      )) As properties
   FROM planet_osm_line As lg ) As f )  As fc;
""".format(columns)

Query string for point objects. Since the big number of points extracted, we limit the number of points as 100, which are randomly selected points, in order to get faster response of codes when display the map with object plots. One can remove the *limit 100* statement in the query and see all the points at once.

In [ ]:
query_json_point = """
SELECT row_to_json(fc)
 FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
 FROM (SELECT 'Feature' As type
    , ST_AsGeoJSON(ST_Transform(lg.way, 4326))::json As geometry
    , row_to_json((SELECT l FROM (SELECT {}) As l
      )) As properties
   FROM planet_osm_point As lg order by random() limit 100) As f )  As fc;
""".format(columns)

Send queries to database and save the results in variables.

In [ ]:
cur.execute(query_json_point)
d_points = cur.fetchone()
cur.execute(query_json_line)
d_lines = cur.fetchone()

Connection to database is closed, since we are done with our queries.

In [ ]:
conn.close()

# Create Map

Since we are focusing on Berlin power data, we are centering there in the map.

In [ ]:
map = ipyl.Map(center=[52.52, 13.45], zoom=11, layout=ipyw.Layout(height='600px'))
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

Loading GeoJSON to display extracted data on the map.

In [ ]:
layer_points = ipyl.GeoJSON(data=d_points[0], hover_style={'fill_color': 'red'}, style={'fill_color':'#001010','fill':True})
layer_lines = ipyl.GeoJSON(data=d_lines[0],style={'color':'red'})

Function **action_handler** is used to handle *click* and *hover* mouse actions, to show label properties below the map. 

But we found that ipyleaflet 'on_hover' feature only works for line object and opened an issue on [here](https://github.com/jupyter-widgets/ipyleaflet/issues/174). 

For point object we need to use 'on_click' on the object then label will be shown below the map.

So, to get power relevant information for line objects, it is enough to hover on the interested line, however, for the point objects, one needs to click on the marker to be able to see the relevant information.

In [ ]:
def action_handler(event=None, id=None, properties=None):
    s = ""
    for key, value in properties.items():
        if str(value)!='None':
            s = s +str(key).title() +"--> " + str(value).title() + ";   \t"
    if s=="":
        s = 'No Data'
    label.value = s

Register action handlers to the GeoJSON objects.

In [ ]:
# Hover lines to show label data.
layer_lines.on_hover(action_handler)
# Click points to show label data.
layer_points.on_click(action_handler)

Add created layers to the map.

In [ ]:
map.add_layer(layer_lines)
map.add_layer(layer_points)

# Show the map.

In [ ]:
ipyw.VBox([map, label])